### Arcade

#### Animierte Tiles 

[Video](https://youtu.be/fZhw76DpCUI)

In *Tiled* können wir Tiles animieren. Im Arcade-Programm können wird die Animation übernehmen mit 

    self.scene.update_animation(delta_time)

in der *on_update*-Methode.

In [ ]:
import arcade

PLAYER_MOVEMENT_SPEED = 5
GRAVITY = 1
PLAYER_JUMP_SPEED = 20

class GameView(arcade.View):
    def __init__(self):
        super().__init__()
        layer_options = {"Walls": {"use_spatial_hash": True}}
        tile_map = arcade.load_tilemap('map.tmx',layer_options=layer_options,scaling=0.5)
        self.scene = arcade.Scene.from_tilemap(tile_map)
        self.player = arcade.Sprite(':resources:/images/animated_characters/female_adventurer/femaleAdventurer_idle.png')
        self.player.position = (30, 200)
        self.scene.add_sprite("Player", self.player)

        self.physics_engine = arcade.PhysicsEnginePlatformer(self.player, walls=self.scene.get_sprite_list("Walls"),
                                                         gravity_constant=GRAVITY)
        
        self.background_color = arcade.csscolor.CORNFLOWER_BLUE
        self.jump_sound = arcade.load_sound(":resources:sounds/jump2.wav")
        self.gameover_sound = arcade.load_sound(":resources:sounds/gameover1.wav")

    def on_update(self, delta_time):    
        self.scene.update_animation(delta_time)        

        if arcade.check_for_collision_with_list(self.player, self.scene["Water"]):
            arcade.play_sound(self.gameover_sound)
            self.player.change_y = -0.1  # Stop gravity

        if self.player.bottom < -20:
            self.player.position = (30, 400)  # Reset player position
       

        self.physics_engine.update()
    def on_draw(self):
        self.clear()
        self.scene.draw()     

    def on_key_press(self, key, modifiers):
        if key == arcade.key.UP:
            if self.physics_engine.can_jump():
                self.player.change_y = PLAYER_JUMP_SPEED
                arcade.play_sound(self.jump_sound)

        if key == arcade.key.LEFT:
            self.player.change_x = -PLAYER_MOVEMENT_SPEED
        elif key == arcade.key.RIGHT:
            self.player.change_x = PLAYER_MOVEMENT_SPEED

    def on_key_release(self, key, modifiers):
        if key == arcade.key.LEFT:
            self.player.change_x = 0
        elif key == arcade.key.RIGHT:
            self.player.change_x = 0

window = arcade.Window(600,400,title="Arcade") 
window.set_location(900, 100)  
game = GameView()
window.show_view(game)
arcade.run()

#### Moving Platforms

[Video](https://youtu.be/oWHjVNM7v9o)

Wenn wir Sprites mit den Attributen 

    boundary_left
    boundary_right
    change_x

der PhysicsEnginePlatformer über den Parameter *platforms=* in einer Spritelist übergeben, dann sorgt die engine für die Hin- und Herbewegung.
Das genaue Mitfahren auf der Platform müssen wir im Code programmieren.





In [ ]:
import arcade

PLAYER_MOVEMENT_SPEED = 5
GRAVITY = 1
PLAYER_JUMP_SPEED = 15

class GameView(arcade.View):
    def __init__(self):
        super().__init__()
        layer_options = {"Walls": {"use_spatial_hash": True}}
        tile_map = arcade.load_tilemap('map.tmx',layer_options=layer_options,scaling=0.5)
        self.scene = arcade.Scene.from_tilemap(tile_map)
        self.player = arcade.Sprite(':resources:/images/animated_characters/female_adventurer/femaleAdventurer_idle.png')
        
        self.player.position = (30, 200)
        self.scene.add_sprite("Player", self.player)
        self.scene.add_sprite_list("Platforms")        
        
        self.jump_sound = arcade.load_sound(":resources:sounds/jump2.wav")

        platform_texture = arcade.load_texture(':resources:/images/tiles/grassHalf.png')
        platforms_data = [(100, 200, 30, 300, 1),(200 ,400 ,100, 500, 1.5)]
        for x, y, left, right, speed in platforms_data:
            sprite = arcade.Sprite(platform_texture, center_x=x, center_y=y,scale=0.5)
            sprite.boundary_left = left
            sprite.boundary_right = right
            sprite.change_x = speed
            self.scene.add_sprite("Platforms", sprite)

        self.background_color = arcade.csscolor.CORNFLOWER_BLUE
        self.physics_engine = arcade.PhysicsEnginePlatformer(self.player, walls=self.scene["Walls"],
                                                             platforms=self.scene["Platforms"],
                                                             gravity_constant=GRAVITY)

    def on_update(self, delta_time):    

        self.scene.update(delta_time)   
        for platform in self.scene['Platforms']:
            if player_stands_on(self.player, platform):
                self.player.center_x += platform.change_x
        self.physics_engine.update()

    def on_draw(self):
        self.clear()
        self.scene.draw()         

    def on_key_press(self, key, modifiers):
        if key == arcade.key.UP:
            if self.physics_engine.can_jump():
                self.player.change_y = PLAYER_JUMP_SPEED
                arcade.play_sound(self.jump_sound)

        if key == arcade.key.LEFT:
            self.player.change_x = -PLAYER_MOVEMENT_SPEED
        elif key == arcade.key.RIGHT:
            self.player.change_x = PLAYER_MOVEMENT_SPEED

    def on_key_release(self, key, modifiers):
        if key == arcade.key.LEFT:
            self.player.change_x = 0
        elif key == arcade.key.RIGHT:
            self.player.change_x = 0


def player_stands_on(player, platform):
    return platform.left < player.right and player.left < platform.right \
        and abs(player.bottom - platform.top) < 5

window = arcade.Window(600,600,title="Arcade") 
window.set_location(900, 100)  
game = GameView()
window.show_view(game)
arcade.run()

#### Leiter

Die *PhysicsEnginePlatformer* hat einen eigenen Parameter für Leiter-Sprites. Leiter sind Sprites, auf denen sich unser Player mit den Pfeiltasten bewegen kann. Die Gravitation ist ausgesetzt. 

[Video](https://youtu.be/5EzimXNcXHg)


In [ ]:
import arcade

PLAYER_MOVEMENT_SPEED = 5
GRAVITY = 1
PLAYER_JUMP_SPEED = 10

class GameView(arcade.View):
    def __init__(self):
        super().__init__()

        # background
        self.background_color = arcade.csscolor.CORNFLOWER_BLUE

        # sounds
        self.jump_sound = arcade.load_sound(":resources:sounds/jump2.wav")

        # walls and ladders
        layer_options = {"Walls": {"use_spatial_hash": True}}
        tile_map = arcade.load_tilemap('map.tmx',layer_options=layer_options,scaling=0.5)
        self.scene = arcade.Scene.from_tilemap(tile_map)

        # player
        self.player = arcade.Sprite(':resources:/images/animated_characters/female_adventurer/femaleAdventurer_idle.png')
        self.player.position = (30, 200)
        self.scene.add_sprite("Player", self.player)

        # platforms
        self.scene.add_sprite_list("Platforms")        
        platform_texture = arcade.load_texture(':resources:/images/tiles/grassHalf.png')
        platforms_data = [(5*64, 2*64, 6*64, 1),(13*64 ,3*64, 9*64, 1.5)]
        for y, left, right, speed in platforms_data:
            sprite = arcade.Sprite(platform_texture, center_x=left, center_y=y,scale=0.5)
            sprite.boundary_left = left
            sprite.boundary_right = right
            sprite.change_x = speed
            self.scene.add_sprite("Platforms", sprite)

        # physics and camera
        self.physics_engine = arcade.PhysicsEnginePlatformer(self.player, walls=self.scene["Walls"],
                                                             platforms=self.scene["Platforms"],
                                                             ladders=self.scene["Ladders"],
                                                             gravity_constant=GRAVITY)
        self.camera = arcade.Camera2D()

    def on_update(self, delta_time):    

        # scene
        self.scene.update(delta_time)   

        # player on platform 
        for platform in self.scene['Platforms']:
            if player_stands_on(self.player, platform):
                self.player.center_x += platform.change_x

        # physics and camera
        self.camera.position = self.player.position
        self.physics_engine.update()

    def on_draw(self):
        self.clear()
        self.camera.use()
        self.scene.draw()         

    def on_key_press(self, key, modifiers):
        
        if key == arcade.key.UP:
            if self.physics_engine.is_on_ladder():
                self.player.change_y = PLAYER_MOVEMENT_SPEED
            elif self.physics_engine.can_jump():
                self.player.change_y = PLAYER_JUMP_SPEED
                arcade.play_sound(self.jump_sound)

        elif key == arcade.key.DOWN:
            if self.physics_engine.is_on_ladder():
                self.player.change_y = -PLAYER_MOVEMENT_SPEED

        if key == arcade.key.LEFT:
            self.player.change_x = -PLAYER_MOVEMENT_SPEED
        elif key == arcade.key.RIGHT:
            self.player.change_x = PLAYER_MOVEMENT_SPEED

    def on_key_release(self, key, modifiers):

        if key == arcade.key.UP:
            if self.physics_engine.is_on_ladder():
                self.player.change_y = 0
        elif key == arcade.key.DOWN:
            if self.physics_engine.is_on_ladder():
                self.player.change_y = 0

        if key == arcade.key.LEFT:
            self.player.change_x = 0
        elif key == arcade.key.RIGHT:
            self.player.change_x = 0


def player_stands_on(player, platform):
    return platform.left < player.right and player.left < platform.right \
        and abs(player.bottom - platform.top) < 5

window = arcade.Window(600,600,title="Arcade") 
window.set_location(900, 100)  
game = GameView()
window.show_view(game)
arcade.run()